In [1]:
import keras_bitcoin
import dataset
import collections
import pandas as pd
from itertools import permutations, combinations
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [236]:
#df = pd.read_csv("../Data/LDA_Data/reduceData3.csv")
#df = dataset.put_on_label_dataset(df, "day", binary=False, values = [-0.5, -0.2, 0.2, 0.5])

df = dataset.get_labeled_dataset(number_of_file=200, binary= False, values = [-5, -0.2, 0.2, 5])
print(collections.Counter(df["label"]))

#df = dataset.put_on_label_dataset(df, "date")
df.head()
print(df.shape)
print(df.head())

Number of files loaded :  202
Counter({4: 6788, 2: 5873, 5: 1374, 1: 950, 3: 610})
(15595, 8)
                                                text  label        date  \
0   I read it, but what exactly is the problem ?\r\n      4  2015-12-17   
1  I like that idea because it increases personal...      4  2015-12-17   
2  When i see a duplicate post i just downvote an...      4  2015-12-17   
3  > P.S. I agree with your sentiment that to be ...      4  2015-12-17   
4  And take money away from Reddit. Doubt it.\r\n...      4  2015-12-30   

   score  nb_replies  stickied  label_m1  label_m2  
0      1           0     False         4         2  
1      1           0     False         4         2  
2      1           0     False         4         2  
3      1           1     False         4         2  
4     14           1     False         4         4  


In [237]:
texts = df["text"]
labels = df["label"]
texts_train, texts_test , labels_train, labels_test, vocab_length, max_sentence_size\
= keras_bitcoin.get_train_test_data(texts, labels)

In [238]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

xg_class = XGBClassifier(objective ='reg:tweedie',
                         max_depth = 9,
                         n_estimators = 100,
                         eta = 0.5,
                         booster = "gblinear")
xg_class.fit(texts_train,labels_train)
preds = xg_class.predict(texts_test)
accuracy = accuracy_score(labels_test, preds)
print(accuracy)

KeyboardInterrupt: 

In [234]:
from sklearn.metrics import mean_squared_error
mean_squared_error(labels_test, preds)

6.768515549855723

In [235]:
from sklearn.metrics import mean_squared_error
import random

mean_squared_error(labels_test, [random.randint(1, 5) for x in labels_test])

5.470663674254569

In [17]:
objectives = ["reg:tweedie", "reg:gamma", "rank:map", "rank:ndcg", "rank:pairwise", "reg:squarederror", "reg:logistic", "binary:logistic", "binary:logitraw", "binary:hinge"]
n_estimators = [100] #[10, 70, 100, 140, 200, 300, 500]
boosters = ["gblinear"] #["gblinear", "gbtree"] # ["gblinear", "gbtree", "dart"]
etas = [0.5]  #[0.2, 0.5, 0.8]
max_depths = [10] #[0, 10, 100, 1000]

In [18]:
print(len(objectives) * len(n_estimators) * len(boosters) * len(etas) * len(max_depths))

10


In [19]:

best = {"eta" : 0, "n_est" : 0, "booster" : 0, "objective" : 0, "depth" : 0, "acc" : 0}
count = 0
for obj in objectives:
    for n_est in n_estimators:
        for b in boosters:
            for eta in etas:
                for depth in max_depths:
                    count += 1
                    print(count)
                    xg_class = XGBClassifier(
                        objective =obj,
                        max_depth = depth,
                        n_estimators = n_est,
                        eta = eta,
                        booster = b)
                    
                    xg_class.fit(texts_train,labels_train)
                    preds = xg_class.predict(texts_test)
                    accuracy = accuracy_score(labels_test, preds)
                    if accuracy > best["acc"]:
                        print(accuracy)
                        best["objective"] = obj
                        best["booster"] = b                        
                        best["eta"] = eta                        
                        best["depth"] = depth                        
                        best["n_est"] = n_est     
                        best["acc"] = accuracy

1
0.3252032520325203
2
3
4
5


KeyboardInterrupt: 

In [162]:
best

{'eta': 0.2,
 'n_est': 10,
 'booster': 'gblinear',
 'objective': 'reg:tweedie',
 'depth': 0,
 'acc': 0.7931570762052877}

In [35]:
epochs=5
batch_size=None
perfs = {}
dropouts = []


#reducedata2
#function_list = ['tanh', 'softmax', 'softsign', 'hard_sigmoid']
#function_list =  list(combinations(function_list, 3))
#function_list = ['softmax', 'softplus', 'sigmoid', 'hard_sigmoid', 'exponential']
#function_list =  list(combinations(function_list, 3))



#stem only
function_list = [['softplus', 'relu', 'sigmoid']]
#no dropout
#function_list = [['tanh', 'softmax'], ['tanh', 'softplus'], ['tanh', 'sigmoid'], ['tanh', 'hard_sigmoid'], ['tanh', 'exponential'], ['softmax', 'sigmoid'], ['softmax', 'hard_sigmoid'], 'softmax', 'exponential'], ['softplus', 'softsign'], ['softplus', 'sigmoid'], ['softplus', 'hard_sigmoid'], ['softplus', 'exponential'], ['softsign', 'sigmoid'], ['softsign', 'hard_sigmoid'], ['softsign', 'exponential'], ['sigmoid', 'hard_sigmoid'], ['hard_sigmoid', 'exponential']]
#function_list =  list(combinations(function_list, 2))
temp = []


#function_list =  list(combinations(["softmax", "softplus", "softsign", "sigmoid", "hard_sigmoid"], ))
#scrambled_list = [ 'tanh', 'softmax' ,'softplus', 'sigmoid', 'hard_sigmoid', 'exponential','linear']
#random.shuffle(scrambled_list)
#function_list =  list(combinations(scrambled_list, 3))
#function_list =  list(combinations(keras_bitcoin.available_activation_functions, 2))

size = len(function_list)
count = 0
for function in function_list:
    count += 1
    print(count, size)
    if isinstance(function, str):
        function = [function]
    else :
        function = list(function)
        dropouts = [0.1, 0.2, 0.1, 0.2]
    model = keras_bitcoin.get_model(texts_train, labels_train, vocab_length,\
                                    max_sentence_size, epochs=epochs,\
                                    batch_size=batch_size, activations_functions=function,\
                                    dropouts=dropouts, verbose = 0)
    loss, accuracy = model.evaluate(texts_test, labels_test, verbose=0)
    if accuracy >= 0.5:
        temp.append(function)
    perfs["_".join(function)] = [accuracy]

1 1


C:\Users\33777\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [36]:
perfs

{'softplus_relu_sigmoid': [0.07726835459470749]}

In [19]:
df_test = dataset.get_labeled_dataset(number_of_file = 10, from_date = "2017-02-10", date_included = False)

Number of files loaded :  10


In [20]:
preds = keras_bitcoin.get_predictions(list(df_test["text"]), model, vocab_length, max_sentence_size)
df_test["preds"] = list(map(lambda x : int(x), preds))
df_test["correct"] = np.equal(preds, df_test["label"])

In [21]:
dataset.get_prediction_stats(df_test)

Number Correct/Wrong Guess : 5/363
              Accuracy : 1.358695652173913
Invalid sentences count 0


In [ ]:
scrambled_list = list(random.shuffle([ 'tanh', 'softmax' ,'softplus', 'sigmoid', 'hard_sigmoid', 'exponential','linear']))


In [14]:
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]
confusion_matrix(y_true, y_pred)

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]], dtype=int64)